In [170]:
import numpy as np
import pandas as pd
import requests 
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

from urllib.request import urlopen
import urllib

from selenium import webdriver
import openpyxl
import time
import math
from collections import Counter


from tqdm.notebook import tqdm

import time
import datetime

In [3]:
def url_split(url):
    main_,para_1 = url.split("?")
    para_2 = para_1.split("&")
    para_3 = [d.replace("=",":").split(":") for d in para_2 if len(d.replace("=",":").split(":")) != 1]
    params_ = {}
    #print(para_3)
    for key,value in para_3:
        params_[key] = value
    return main_,params_

In [14]:
url_1 = "https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid=469&date=20230724&page=2"

In [15]:
main_1,para_1 = url_split(url_1)
para_1

{'mode': 'LPOD', 'mid': 'sec', 'oid': '469', 'date': '20230724', 'page': '2'}

In [16]:
main_1

'https://news.naver.com/main/list.naver'

In [17]:
journal_cate = {"경향신문":"032","국민일보":"005","동아일보":"020",
                "문화일보":"021","서울신문":"081","세계일보":"022","조선일보":"023","중앙일보":"025",
               "한겨례":"028","한국일보":"469"}

In [130]:
main_1 = 'https://news.naver.com/main/list.naver'

In [181]:
def crawling_all(Main_1,jou,date):
    user_agent = UserAgent()
    headers = {'User-Agent': user_agent.random}
    paras = {'mode': 'LPOD', 'mid': 'sec'}
    paras["oid"] = jou
    paras["date"] = date
    dup_check = ""
    df_all = pd.DataFrame()
    tq = tqdm(range(20))
    for i in tq:
        paras["page"] = str(i+1)
        main_1 = Main_1 + "?"
        for key,value in paras.items():
            sort_ = key + "=" + value + "&"
            main_1 += sort_
        
        html = requests.get(main_1,headers=headers).text
        soup = BeautifulSoup(html,"html.parser")
        #print(main_1)
        try:
            dup_c = [i["alt"] for i in soup.select(".type06_headline img")][0]
            if dup_c != dup_check:
                dup_check = dup_c
                title_list,href_list,brief_list = [],[],[]

                for tag in soup.select(".type06_headline li dl"):
                    try:

                        title_list.append(tag.find("dt").find("a").find("img")["alt"])
                        href_list.append(tag.find("dt").find("a")["href"])
                        brief_list.append(tag.find("dd").find("span").text)
                    except:
                        title_list.append(tag.find("dt").find("a").text.strip())
                        href_list.append(tag.find("dt").find("a")["href"])
                        brief_list.append(tag.find("dd").find("span").text)
                text_list = []    

                for test_href in href_list:
                    url_sub = requests.get(test_href,headers=headers).text
                    soup_sub = BeautifulSoup(url_sub,"html.parser")
                    text_ = ""
                    for tag2 in soup_sub.select("#dic_area"):
                        text_ += tag2.text.strip().replace("\n"," ")
                    text_list.append(text_)
                df_ = pd.DataFrame([title_list,href_list,text_list]).T
                df_out = df_[df_[2]!=""].reset_index(drop=True)
                df_all = pd.concat([df_all,df_out]).reset_index(drop=True)
            else:
                break
        except:
            pass

    return df_all

In [154]:
DF_ = crawling_all(main_1,"032","20230724")
DF_

  0%|          | 0/20 [00:00<?, ?it/s]

https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid=032&date=20230724&page=1&
https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid=032&date=20230724&page=2&
https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid=032&date=20230724&page=3&
https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid=032&date=20230724&page=4&
https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid=032&date=20230724&page=5&
https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid=032&date=20230724&page=6&
https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid=032&date=20230724&page=7&
https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid=032&date=20230724&page=8&


,0,1,2
0,"올해 상반기 개선된 제조업 자금 사정, 주원인은 ‘차입금 증가’",https://n.news.naver.com/mnews/article/032/000...,주요 제조기업 ‘자금사정 현황’ 및 ‘자금조달 방식’ 응답. 전경련 제공올해 상반기...
1,비비큐 “K-푸드 세계 알리는 민간 외교관 될 터”,https://n.news.naver.com/mnews/article/032/000...,윤홍근 회장 ‘글로벌 치킨캠프’ 직접 챙겨세계 각국 대사 등 초청 한식문화도 소개국...
2,"“새집 줄게 헌집 다오” 제주개발공사, 기존주택 매입사업 탄력",https://n.news.naver.com/mnews/article/032/000...,"제주개발공사, 상반기 98호 이어 62호 매입주거취약계층에 저렴한 가격 임대 ..."
3,“안전에 대한 믿음과 희망 지켜달라” 헌재에 보내는 이태원 참사 유가족의 편지,https://n.news.naver.com/mnews/article/032/000...,이상민 행정안전부 장관에 대한 헌법재판소 탄핵 심판을 하루 앞둔 24일 이태원참사 ...
4,"손석구, ‘가짜 연기’ 발언 비판한 남명렬에 손편지 사과",https://n.news.naver.com/mnews/article/032/000...,배우 손석구가 지난달 27일 서울 강서구 LG아트센터서울에서 열린 연극 <나무 위의...
...,...,...,...
65,[김만권의 손길] 진짜 ‘이권 카르텔’은 어디에 있을까,https://n.news.naver.com/mnews/article/032/000...,| 김만권 경희대 학술연구교수·정치철학자윤 대통령이 정의하는 카르텔이 뭔지 명확하지...
66,[우리말 산책] 복숭아를 제사상에 안 올리는 이유,https://n.news.naver.com/mnews/article/032/000...,| 엄민용 기자요즘처럼 무더운 날에는 입맛을 잃기 쉽다. 이런 때 많이 찾는 것이 ...
67,[기고] 국민노후 저버린 윤석열 정부의 ‘이권 카르텔’,https://n.news.naver.com/mnews/article/032/000...,| 김보영 영남대 휴먼서비스학과 교수지난 19일 보건복지부 등이 주최한 ‘신노년층을...
68,[노래와 세상] 비에 젖은 마음,https://n.news.naver.com/mnews/article/032/000...,| 오광수 시인·대중음악평론가 비는 사랑을 부른다. 적어도 노래 속에서는 그렇...


In [178]:
start_date = datetime.datetime.strptime("20220101","%Y%m%d")

journ_ = journal_cate["경향신문"]
DF_OUT = pd.DataFrame()
for i in range(30):
    date_ = start_date + datetime.timedelta(days=1)
    date_ = str(date_.strftime("%Y%m%d"))
    print(date_)
    df_can = crawling_all(main_1,journ_,date_)
    DF_OUT = pd.concat([DF_OUT,df_can])
    start_date = datetime.datetime.strptime(date_,"%Y%m%d")

20220102


  0%|          | 0/20 [00:00<?, ?it/s]

20220103


  0%|          | 0/20 [00:00<?, ?it/s]

20220104


...

20220117


  0%|          | 0/20 [00:00<?, ?it/s]

20220118


  0%|          | 0/20 [00:00<?, ?it/s]

IndexError: list index out of range

In [180]:
DF_OUT.to_excel(r"C:\Users\Samsung\Desktop\all\Project\23_Inj\Data\out\sample_2.xlsx")